<a href="https://colab.research.google.com/github/ea-Mitsuoka/ionicons/blob/master/bqml_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```
---

# 概要
このノートブックでは、BigQuery MLのAutoMLでLTV予測を行うモデル作成を行っています。


### 費用

このチュートリアルでは、Google BigQuery および BigQuery ML を使います。金額について詳細は  [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing) [Pricing
Calculator](https://cloud.google.com/products/calculator/)
を参照ください。

### 事前準備

**このノートブックを実行するには事前に次の手順が必要です。**

1. [GCP プロジェクトの作成もしくは選択](https://console.cloud.google.com/cloud-resource-manager). 最初にアカウントを作成すると、300ドルの無料クレジットが付与されます。

2. [課金設定を有効にします](https://cloud.google.com/billing/docs/how-to/modify-project)

3.   [BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) を有効にします。

4. 下のセルを実行し、Google アカウントでログインして表示されるキーをセルに入力します。

5. 「プロジェクト変数の設定」のセルにご自身のプロジェクトIDを入力してください。

In [ ]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
auth.authenticate_user()

In [ ]:
#@title プロジェクト変数の設定 { run: "auto", display-mode: "form" }
project_id = '' #@param {type:"string"}
#dataset_name = "ga" #@param {type:"string"}
client = bigquery.Client(project=project_id)

まず、モデルを保存するためのデータセットが必要になります。 （エラーが発生した場合は、BigQueryコンソールから"automl"というデータセットを作成してください）。

In [ ]:
%%bash -s "$project_id"

gcloud config set project $1
bq --location=US mk -d automl

Process is terminated.


## データセット読み取り
このセクションでは、BigQueryでLTV分析用のデータセットをクエリしています。

In [ ]:
query = '''
SELECT * from ltv.ltv_features
'''
df = client.query(query).to_dataframe()

In [ ]:
df.head()

,customer_id,monetary,frequency,recency,T,time_between,avg_basket_value,avg_basket_size,cnt_returns,has_returned,target_monetary
0,17198,521.66,2,6,248,3.0,260.83,121.0,0,0,692.22
1,18180,601.85,2,6,19,3.0,300.92,276.5,0,0,1843.75
2,13884,189.95,2,6,160,3.0,94.97,31.0,0,0,781.30
3,14731,251.02,2,7,89,3.5,125.51,91.0,0,0,380.02
4,15467,402.56,2,7,29,3.5,201.28,117.5,0,0,511.88


トレーニング データを確認できたので、 AutoML で target_monetary 列を予測するモデルを作成します。
target_monetary 列は金額で構成されているため、回帰モデルでモデル作成を行います。

In [ ]:
query = '''
CREATE OR REPLACE MODEL automl.ltv
OPTIONS(input_label_cols=['target_monetary'], 
        model_type='automl_regressor')
AS
SELECT * EXCEPT(customer_id)
FROM ltv.ltv_features
'''
client.query(query)

モデルを作成したら、予測を試します。

In [ ]:
query = """
SELECT * FROM ML.PREDICT(MODEL automl.ltv, (
    SELECT * EXCEPT(customer_id) FROM `ltv.ltv_features` LIMIT 100))
  """
df = client.query(query).to_dataframe()
df

### クリーンアップ
すべての作業が完了したら、以下のURLを参考に、プロジェクトを必要に応じてクリーンアップしてください。
https://cloud.google.com/go/getting-started/delete-tutorial-resources?hl=ja